In [7]:
import numpy as np
import cv2
import dlib
from scipy.spatial import distance
import os
import pandas as pd
##
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('.\shape_predictor_68_face_landmarks.dat')
# predictor = dlib.shape_predictor(r'C:\Users\USER\anaconda3\Lib\site-packages\face_recognition_models\models\shape_predictor_5_face_landmarks.dat')
font = cv2.FONT_HERSHEY_SIMPLEX            # 利用cv2.putText輸出1-68

#寫一個函數輸入檔案，產出一行五特徵分數+總分
def scoring(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # 取灰度

    rects = detector(img_gray, 0)             # 人臉數rects
    pos1 = []
    for i in range(len(rects)):
        landmarks = np.matrix([[p.x, p.y] for p in predictor(img,rects[i]).parts()])
        for idx, point in enumerate(landmarks):

            pos = (point[0, 0], point[0, 1])   # 68點的座標
            #print(idx,pos)
            pos1.append(pos)
            #print(pos1)
            cv2.circle(img, pos, 5, color=(0, 255, 0)) # 利用cv2.circle給每個特徵點畫一個圈，共68個
            cv2.putText(img, str(idx+1), pos, font, 0.8, (0, 0, 255), 1,cv2.LINE_AA)
    #score = 0
    #print(distance.euclidean(pos1[8],pos1[33]))
    
    #第一指標"鼻樑=鼻頭到下巴底"
    eb1 = (pos1[21])
    eb2 = (pos1[22])
   
    eb_m = ((eb1[0]+eb2[0])/2,(eb1[1]+eb2[1])/2) #眉間位置
    dis_1=distance.euclidean(eb_m,pos1[33]) #鼻樑
    dis_2=distance.euclidean(pos1[8],pos1[33]) #鼻頭到下巴
    rate_01 = (dis_1)/(dis_2) #比例
    #print(rate_01)

    #第二指標"眉毛=臉寬/5"
    dis_3= distance.euclidean(pos1[0],pos1[16])/5 #臉寬/5
    dis_4= distance.euclidean(pos1[21],pos1[22]) #眉毛距離
    rate_02= (dis_3)/(dis_4) #比例
    #print(rate_02)
        
    #第三指標"眼距=右眼寬"
    dis_5 = distance.euclidean(pos1[39],pos1[42]) #眼距
    dis_6 = distance.euclidean(pos1[36],pos1[39]) #右眼寬
    rate_03= (dis_5)/(dis_6)
    #print(rate_03)
    
    #第四指標"眼距=左眼寬"
    dis_7 = distance.euclidean(pos1[42],pos1[45])#左眼寬
    rate_04= (dis_5)/(dis_7)
    #print(rate_04)
    
    #第五指標"嘴寬=鼻寬*1.6"
    dis_8 = (distance.euclidean(pos1[48],pos1[54]))#嘴寬
    dis_9 = (distance.euclidean(pos1[31],pos1[35]))*1.6#鼻寬*1.6    
    rate_05= (dis_8)/(dis_9)
    #print(rate_05)
    
    #總分
    total_score=rate_01+rate_02+rate_03+rate_04+rate_05
    new_row=pd.DataFrame([[filename, rate_01, rate_02,rate_03,rate_04,rate_05,total_score]], columns=df.columns.tolist() )
    return new_row

In [8]:



#numpy
total_array=np.zeros([1,6])

#pandas
df = pd.DataFrame(np.empty((1,7)), columns=["filename","par1","par2","par3","par4","par5","total"])
#迴圈
for filename in os.listdir("male"):
    
    img = cv2.imread("./male/"+filename)     # cv2讀取影像   
    '''
    #numpy
    array_sample=np.array([rate_01,rate_02,rate_03,rate_04,rate_05,total_score]).reshape(1,6)
    total_array=np.append(array_sample,total_array , axis=0)
    '''
    #pandas 
    #執行辨識+打分函數
    new_row=scoring(img)
    
    #黏到總表上
    df = df.append(new_row, ignore_index=True)
    

    #cv2.imshow('img', img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
'''
print("numpy方法\n",total_array)
'''
df = df.drop([0], axis=0)
df = df.reset_index(drop=True)
print("pandas方法\n",df)
df.to_csv("ratiosm.csv")

pandas方法
       filename      par1      par2      par3      par4      par5     total
0    E (1).jpg  1.107682  1.816468  1.534294  1.547825  1.151139  7.157407
1   E (10).jpg  1.187684  1.117882  1.667767  1.666392  1.238934  6.878659
2   E (11).jpg  0.912425  1.688724  1.467014  1.525764  1.322879  6.916806
3    E (2).jpg  1.108896  1.403471  1.578061  1.578061  1.043267  6.711757
4    E (3).jpg  1.071261  1.357410  1.724835  1.588420  1.201448  6.943374
5    E (4).jpg  1.137529  1.337697  1.561524  1.635517  1.126457  6.798724
6    E (5).jpg  0.984202  1.116791  1.653796  1.810715  1.088209  6.653713
7    E (6).jpg  1.006099  1.405748  1.495799  1.376042  1.161324  6.445011
8    E (7).jpg  0.940066  1.350130  1.666024  1.666667  1.169597  6.792484
9    E (8).jpg  0.915574  1.347033  1.493908  1.490712  1.216624  6.463850
10   E (9).jpg  1.189737  1.138060  1.585386  1.603122  1.220509  6.736813
11   W (1).jpg  0.828542  1.457321  1.510835  1.437315  1.119261  6.353274
12  W (10).jpg 

In [5]:
#找欄位平均值
meanscore_list=df.mean()[:5].tolist()
print(meanscore_list)

#total存list
total_list=df["total"].tolist()
print(total_list)

NameError: name 'df' is not defined

## 找最像的臉孔

In [6]:
import numpy as np
import cv2
import dlib
from scipy.spatial import distance
import os
import pandas as pd
def resize (img):
    #取得長寬做統一修改
    w_ori,h_ori= img.shape[:2]
    new_w=500
    new_h=int((new_w*h_ori)/w_ori)
    print(w_ori,h_ori)
    dst = cv2.resize(img, (new_h, new_w), interpolation=cv2.INTER_AREA)
    return dst

#讀檔案
for filename in os.listdir("test"):
    df2=pd.read_csv('./ratiosm.csv') #打開資料庫
    
    tar_img = cv2.imread("./test/"+filename)
    tar_row=scoring(tar_img)
    num=float(tar_row["total"])
    print(num)
    #找相似照片
    df2["scoring"]=np.nan;
    for i in range(df2.shape[0]):
        df2["scoring"][i]=abs(df2["total"][i]-num)
    print(df2)
    
    mins = df2["scoring"].idxmin() #找到索引 

    print(mins)
    filename=df2["filename"][mins]#找到檔案名
    print(filename)
    #打開圖片
    img2=cv2.imread("./male/"+filename)
    #修改原始圖片和目標圖片
    dst_origin=resize(tar_img)
    dst_compare=resize(img2)
    
    cv2.imshow('origin',dst_origin)
    cv2.imshow('compare',dst_compare)
    cv2.waitKey(0)

    cv2.destroyAllWindows()

KeyError: 'total'

In [13]:
#視訊功能
import cv2
 
cap = cv2.VideoCapture(0)
 
i=0
save_path = './test/'      # 儲存路徑(可自行修改)
file_name = 'test'                          # 檔名(可自行修改)
 
while(1):
    ret, frame = cap.read()                 # 讀取相機影像
    cv2.imshow('cap', frame)                # 顯示影像
 
    n=cv2.waitKey(1)
 
    if n == ord('q'):
        break
    elif n == ord('s'):
        cv2.imwrite(save_path + file_name + '_' + str(i) + '.jpg',frame)
        print('save:',file_name + '_' + str(i) + '.jpg')
        i = i + 1
 
 
cap.release()                                  #釋放鏡頭
cv2.destroyAllWindows()                        #關閉所有視窗